In [1]:
from __future__ import print_function
import os
import numpy as np
from skimage.io import imsave, imread
data_path = 'data/nso/'
image_rows = 420
image_cols = 580

In [2]:
train_data_path = os.path.join(data_path, 'gray/train')
images = os.listdir(train_data_path)
total = len(images) / 2
imgs = np.ndarray((int(total), image_rows, image_cols), dtype=np.uint8)
imgs_mask = np.ndarray((int(total), image_rows, image_cols), dtype=np.uint8)
i = 0

In [3]:
for image_name in images:
        if 'mask' in image_name:
            continue
        image_mask_name = image_name.split('.')[0] + '_mask.png'
        img = imread(os.path.join(train_data_path, image_name), as_grey=True)
        img_mask = imread(os.path.join(train_data_path, image_mask_name), as_grey=True)

        img = np.array([img])
        img_mask = np.array([img_mask])

        imgs[i] = img
        imgs_mask[i] = img_mask

        if i % 100 == 0:
            print('Done: {0}/{1} images'.format(i, total))
        i += 1
print('Loading done.')

Done: 0/1000.0 images
Done: 100/1000.0 images
Done: 200/1000.0 images
Done: 300/1000.0 images
Done: 400/1000.0 images
Done: 500/1000.0 images
Done: 600/1000.0 images
Done: 700/1000.0 images
Done: 800/1000.0 images
Done: 900/1000.0 images
Loading done.


In [4]:
np.save(data_path+'imgs_train.npy', imgs)
np.save(data_path+'imgs_mask_train.npy', imgs_mask)
print('Saving to .npy files done.')

Saving to .npy files done.


In [5]:
train_data_path = os.path.join(data_path, 'gray/test')
images = os.listdir(train_data_path)
total = len(images)
imgs = np.ndarray((total, image_rows, image_cols), dtype=np.uint8)
imgs_id = np.ndarray((total, ), dtype=np.int32)
i = 0

In [6]:
for image_name in images:
        img_id = int(image_name.split('.')[0])
        img = imread(os.path.join(train_data_path, image_name), as_grey=True)

        img = np.array([img])

        imgs[i] = img
        imgs_id[i] = img_id

        if i % 100 == 0:
            print('Done: {0}/{1} images'.format(i, total))
        i += 1
print('Loading done.')

Done: 0/274 images
Done: 100/274 images
Done: 200/274 images
Loading done.


In [7]:
np.save(data_path+'imgs_test.npy', imgs)
np.save(data_path+'imgs_id_test.npy', imgs_id)
print('Saving to .npy files done.')

Saving to .npy files done.


In [8]:
from __future__ import print_function
import os
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K

Using TensorFlow backend.


In [9]:
imgs_train = np.load(data_path+'imgs_train.npy')
imgs_mask_train = np.load(data_path+'imgs_mask_train.npy')

In [10]:
img_rows = 96
img_cols = 96
smooth = 1.

In [11]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [12]:
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [13]:
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)
    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p

In [14]:
inputs = Input((img_rows, img_cols, 1))
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

model = Model(inputs=[inputs], outputs=[conv10])

model.compile(optimizer=Adam(lr=1e-5), loss=dice_coef_loss, metrics=[dice_coef])

In [15]:
imgs_train = preprocess(imgs_train)
imgs_mask_train = preprocess(imgs_mask_train)

imgs_train = imgs_train.astype('float32')
mean = np.mean(imgs_train)  # mean for data centering
std = np.std(imgs_train)  # std for data normalization

imgs_train -= mean
imgs_train /= std

imgs_mask_train = imgs_mask_train.astype('float32')
imgs_mask_train /= 255. # scale masks to [0, 1]

C:\ProgramData\Miniconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [16]:
model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss', save_best_only=True)
model.fit(imgs_train, imgs_mask_train, batch_size=32, nb_epoch=20, verbose=1, shuffle=True, validation_split=0.2, callbacks=[model_checkpoint])

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 800 samples, validate on 200 samples
Epoch 1/20
800/800 [==============================] - 144s - loss: -0.0398 - dice_coef: 0.0398 - val_loss: -0.1097 - val_dice_coef: 0.1097
Epoch 2/20
800/800 [==============================] - 143s - loss: -0.0403 - dice_coef: 0.0403 - val_loss: -0.1122 - val_dice_coef: 0.1122
Epoch 3/20
800/800 [==============================] - 142s - loss: -0.0410 - dice_coef: 0.0410 - val_loss: -0.1145 - val_dice_coef: 0.1145
Epoch 4/20
800/800 [==============================] - 143s - loss: -0.0419 - dice_coef: 0.0419 - val_loss: -0.1169 - val_dice_coef: 0.1169
Epoch 5/20
800/800 [==============================] - 142s - loss: -0.0427 - dice_coef: 0.0427 - val_loss: -0.1195 - val_dice_coef: 0.1195
Epoch 6/20
800/800 [==============================] - 142s - loss: -0.0431 - dice_coef: 0.0431 - val_loss: -0.1246 - val_dice_coef: 0.1246
Epoch 7/20
800/800 [==============================] - 143s - loss: -0.0463 - dice_coef: 0.0463 - val_loss: -0.1357 - val

In [17]:
imgs_test = np.load(data_path+'imgs_test.npy')
imgs_test = preprocess(imgs_test)
imgs_test = imgs_test.astype('float32')
imgs_test -= mean
imgs_test /= std

C:\ProgramData\Miniconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [18]:
model.load_weights('weights.h5')

In [19]:
imgs_mask_test = model.predict(imgs_test, verbose=1)
np.save('imgs_mask_test.npy', imgs_mask_test)

274/274 [==============================] - 16s    


In [20]:
imgs_id_test = np.load(data_path+'imgs_id_test.npy')

In [21]:
pred_dir = data_path+'preds'
if not os.path.exists(pred_dir):
    os.mkdir(pred_dir)
for image, image_id in zip(imgs_mask_test, imgs_id_test):
    image = (image[:, :, 0] * 255.).astype(np.uint8)
    imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)

C:\ProgramData\Miniconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: data/nso/preds\103_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\ProgramData\Miniconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: data/nso/preds\104_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\ProgramData\Miniconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: data/nso/preds\105_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\ProgramData\Miniconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: data/nso/preds\106_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\ProgramData\Miniconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: data/nso/preds\177_pred.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\ProgramData\Miniconda3\lib\site-packages\skimage\io\_io.py:132: UserWarning: data/nso/preds\51_pred.png is a low